In [1]:
from __future__ import print_function, division
import matplotlib.pyplot as plt
import pandas as pd

from neuralnilm.data.loadactivations import load_nilmtk_activations
from neuralnilm.data.syntheticaggregatesource import SyntheticAggregateSource
from neuralnilm.data.datapipeline import DataPipeline
from neuralnilm.data.processing import DivideBy, IndependentlyCenter
from neuralnilm.data.datathread import DataThread

In [2]:
NILMTK_FILENAME = '/data/mine/vadeec/merged/ukdale.h5'
TARGET_APPLIANCE = 'kettle'
SEQ_LENGTH = 256
SAMPLE_PERIOD = 6
STRIDE = SEQ_LENGTH
WINDOWS = {
    'train': {
        1: ("2014-01-01", "2014-02-01")
    },
    'unseen_activations_of_seen_appliances': {
        1: ("2014-02-02", "2014-02-08")                
    },
    'unseen_appliances': {
        2: ("2013-06-01", "2013-06-07")
    }
}

LOADER_CONFIG = {
    'nilmtk_activations': dict(
        appliances=['kettle', 'microwave', 'washing machine'],
        filename=NILMTK_FILENAME,
        sample_period=SAMPLE_PERIOD,
        windows=WINDOWS
    )
}



In [3]:
from neuralnilm.data.stridesource import StrideSource

In [4]:
stride_source = StrideSource(
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    filename=NILMTK_FILENAME,
    windows=WINDOWS,
    sample_period=SAMPLE_PERIOD,
    stride=STRIDE
)

INFO:neuralnilm.data.stridesource:Loading NILMTK data...
INFO:neuralnilm.data.stridesource:Loading data for UK-DALE_building_1...
INFO:neuralnilm.data.stridesource:Loaded data from building UK-DALE_building_1 for fold train from 2014-01-01 00:00:06+00:00 to 2014-01-31 23:59:54+00:00.
INFO:neuralnilm.data.stridesource:Loading data for UK-DALE_building_2...
INFO:neuralnilm.data.stridesource:Loaded data from building UK-DALE_building_2 for fold unseen_appliances from 2013-06-01 00:00:00+01:00 to 2013-06-06 23:59:54+01:00.
INFO:neuralnilm.data.stridesource:Loading data for UK-DALE_building_1...
INFO:neuralnilm.data.stridesource:Loaded data from building UK-DALE_building_1 for fold unseen_activations_of_seen_appliances from 2014-02-02 00:00:00+00:00 to 2014-02-07 23:59:54+00:00.
INFO:neuralnilm.data.stridesource:Done loading NILMTK mains data.


In [5]:
nilmtk_activations = load_nilmtk_activations(**LOADER_CONFIG['nilmtk_activations'])

INFO:neuralnilm.data.loadactivations:Loading NILMTK activations...
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 111 kettle activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 119 microwave activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading washing machine for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 23 washing machine activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 kettle activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 30 microwave activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading washing mac

In [6]:
nilmtk_activations.keys()

['train', 'unseen_activations_of_seen_appliances', 'unseen_appliances']

In [7]:
from neuralnilm.data.realaggregatesource import RealAggregateSource

In [8]:
ras = RealAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    filename=NILMTK_FILENAME,
    windows=WINDOWS,
    sample_period=SAMPLE_PERIOD
)

INFO:neuralnilm.data.realaggregatesource:Loading NILMTK mains...
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_1...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_1 for fold train from 2014-01-01 00:00:00+00:00 to 2014-01-31 23:59:54+00:00.
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_2...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_2 for fold unseen_appliances from 2013-06-01 00:00:00+01:00 to 2013-06-06 23:59:54+01:00.
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_1...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_1 for fold unseen_activations_of_seen_appliances from 2014-02-02 00:00:00+00:00 to 2014-02-07 23:59:54+00:00.
INFO:neuralnilm.data.realaggregatesource:Done loading NILMTK mains data.
INFO:neuralnilm.data.realaggregatesource:Found 89 sections without targ

In [9]:
ras.target_inclusion_prob = 0.5
for i in range(50):
    seq = ras.get_sequence()

In [10]:
#fig, axes = plt.subplots(2)
#axes[0].plot(seq.input)
#axes[1].plot(seq.target)
#plt.show()

In [11]:
nilmtk_activations['train']['kettle']['UK-DALE_building_1'][0]._metadata

['name', 'building', 'appliance', 'fold']

In [12]:
source = SyntheticAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    allow_incomplete_target=False,
    sample_period=SAMPLE_PERIOD
)

In [13]:
FOLD = 'train'
#FOLD = 'unseen_activations_of_seen_appliances'
#FOLD = 'unseen_appliances'
seq = source.get_sequence(enable_all_appliances=True)

In [14]:
#all(seq.all_appliances.sum(axis=1) == seq.input[:, 0])

In [15]:
#fig, axes = plt.subplots(2, sharex=True)
#seq.all_appliances.plot(ax=axes[0])
#axes[1].plot(seq.input)
#fig.suptitle(FOLD)
#plt.show()

In [16]:
sample = source.get_batch(num_seq_per_batch=1024).next()

In [17]:
pipeline = DataPipeline(
    sources=[source, ras, stride_source],
    num_seq_per_batch=64,
    input_processing=[DivideBy(sample.before_processing.input.flatten().std()), IndependentlyCenter()],
    target_processing=[DivideBy(sample.before_processing.target.flatten().std())]
)

# Disaggregation

nilmtk_disag_source = NILMTKDisagSource(
    filename=NILMTK_FILENAME,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    buildings=[5],
    window_per_building={},
    stride=STRIDE,
    sample_period=SAMPLE_PERIOD
)

disag_pipeline = deepcopy(pipeline)
disag_pipeline.source = nilmtk_disag_source

disaggregator = Disaggregator(
    pipeline=disag_pipeline,
    output_path=PATH  # 
)

Disagregator ideas:

* make a copy of pipeline but swap source for a NILMTKDisagSource
* NILMTKDisagSource loads all data into memory (?) and iterates over chunks of it (get seq_length from pipeline.source.seq_length)
* 

In [18]:
from lasagne.layers import InputLayer, RecurrentLayer, DenseLayer, ReshapeLayer

def get_net_0(input_shape, target_shape=None):
    NUM_UNITS = {
        'dense_layer_0': 100,
        'dense_layer_1':  50,
        'dense_layer_2': 100
    }

    if target_shape is None:
        target_shape = input_shape
    
    # Define layers
    input_layer = InputLayer(
        shape=input_shape
    )
    
    # Dense layers
    dense_layer_0 = DenseLayer(
        input_layer, 
        num_units=NUM_UNITS['dense_layer_0']
    )
    dense_layer_1 = DenseLayer(
        dense_layer_0, 
        num_units=NUM_UNITS['dense_layer_1']
    )
    dense_layer_2 = DenseLayer(
        dense_layer_1, 
        num_units=NUM_UNITS['dense_layer_2']
    )
    
    # Output
    final_dense_layer = DenseLayer(
        dense_layer_2,
        num_units=target_shape[1] * target_shape[2],
        nonlinearity=None
    )
    output_layer = ReshapeLayer(
        final_dense_layer,
        shape=target_shape
    )
    return output_layer

In [19]:
from neuralnilm.net import Net

batch = pipeline.get_batch()
output_layer = get_net_0(
    batch.after_processing.input.shape, 
    batch.after_processing.target.shape
)
net = Net(output_layer)

In [21]:
from neuralnilm.trainer import Trainer
from neuralnilm.metrics import Metrics

trainer = Trainer(
    net=net,
    data_pipeline=pipeline,
    experiment_id=["4"],
    metrics=Metrics(state_boundaries=[4]),
    requested_learning_rates={0: 1E-2},
    repeat_callbacks=[
        (1000, Trainer.save_params),
        (1000, Trainer.plot_estimates),
        ( 500, Trainer.validate)
    ]
)

In [22]:
report = trainer.report()
report['data']['activations'] = LOADER_CONFIG
from neuralnilm.utils import sanitise_dict_for_mongo
sanitise_dict_for_mongo(report)

{'_id': '4',
 'data': {'activations': {'nilmtk_activations': {'appliances': ['kettle',
     'microwave',
     'washing machine'],
    'filename': '/data/mine/vadeec/merged/ukdale.h5',
    'sample_period': 6,
    'windows': {'train': {'1': ('2014-01-01', '2014-02-01')},
     'unseen_activations_of_seen_appliances': {'1': ('2014-02-02',
       '2014-02-08')},
     'unseen_appliances': {'2': ('2013-06-01', '2013-06-07')}}}},
  'pipeline': {'input_processing': [{'divisor': 610.2794799804688,
     'name': 'DivideBy'},
    {'name': 'IndependentlyCenter'}],
   'num_seq_per_batch': 64,
   'rng_seed': None,
   'source_probabilities': [0.3333333333333333,
    0.3333333333333333,
    0.3333333333333333],
   'sources': {'0': {'allow_incomplete_distractors': True,
     'allow_incomplete_target': False,
     'distractor_inclusion_prob': 0.25,
     'include_incomplete_target_in_output': True,
     'name': 'SyntheticAggregateSource',
     'num_batches_for_validation': 16,
     'rng_seed': None,
     '

In [ ]:
trainer.fit(50000)

INFO:neuralnilm.trainer:Starting training for 50000 iterations.
INFO:neuralnilm.trainer:Iteration 0: Change learning rate to 1.0E-02
INFO:neuralnilm.trainer:Compiling train cost function...


In [ ]:
net.train_iterations